In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

In [2]:
%%capture
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00363/Dataset.zip
!unzip Dataset.zipn

In [3]:
df = pd.read_csv('Dataset/Training/Features_Variant_1.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53
0,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,806.0,7.574627,0.0,69.435826,0.0,76.0,2.604478,0.0,8.505502,0.0,806.0,10.649254,1.0,70.254788,-69.0,806.0,4.970149,0.0,69.85058,0,0,0,0,0,65,166,2,0,24,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,806.0,7.574627,0.0,69.435826,0.0,76.0,2.604478,0.0,8.505502,0.0,806.0,10.649254,1.0,70.254788,-69.0,806.0,4.970149,0.0,69.85058,0,0,0,0,0,10,132,1,0,24,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,806.0,7.574627,0.0,69.435826,0.0,76.0,2.604478,0.0,8.505502,0.0,806.0,10.649254,1.0,70.254788,-69.0,806.0,4.970149,0.0,69.85058,0,0,0,0,0,14,133,2,0,24,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
3,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,806.0,7.574627,0.0,69.435826,0.0,76.0,2.604478,0.0,8.505502,0.0,806.0,10.649254,1.0,70.254788,-69.0,806.0,4.970149,0.0,69.85058,7,0,3,7,-3,62,131,1,0,24,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
4,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,806.0,7.574627,0.0,69.435826,0.0,76.0,2.604478,0.0,8.505502,0.0,806.0,10.649254,1.0,70.254788,-69.0,806.0,4.970149,0.0,69.85058,1,0,0,1,0,58,142,5,0,24,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0


In [4]:
X_train, y_train = df.drop(columns=53), df[53]
X_train.head(), y_train.head()

(       0   1    2   3    4      5          6   ...  46  47  48  49  50  51  52
 0  634995   0  463   1  0.0  806.0  11.291045  ...   0   0   0   0   0   0   1
 1  634995   0  463   1  0.0  806.0  11.291045  ...   0   0   0   0   0   1   0
 2  634995   0  463   1  0.0  806.0  11.291045  ...   0   0   0   0   0   0   1
 3  634995   0  463   1  0.0  806.0  11.291045  ...   0   1   0   0   0   0   0
 4  634995   0  463   1  0.0  806.0  11.291045  ...   0   0   0   1   0   0   0
 
 [5 rows x 53 columns], 0    0
 1    0
 2    0
 3    0
 4    0
 Name: 53, dtype: int64)

In [5]:
from sklearn.preprocessing import StandardScaler

X_train =  pd.DataFrame(StandardScaler().fit_transform(X_train))

In [6]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, epoches=10):
        self.learning_rate = learning_rate
        self.epoches = epoches

    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        self.b = 0
        self.W =np.random.randn(n_features)
    
        for _ in range(self.epoches):
            y_pred = X @ self.W + self.b

            self.b -= self.learning_rate * (2 * np.sum(y_pred - y) / n_samples)
            self.W -= self.learning_rate * (2 * (X.T @ (y_pred - y)) / n_samples)
        
    def predict(self, X):
        return X @ self.W + self.b

In [7]:
def mean(y, y_pred):
    return (y.to_numpy() - y_pred.to_numpy()).mean()

In [8]:
def std(y, y_pred):
    return (y.to_numpy() - y_pred.to_numpy()).std()

In [9]:
def r2_score(y, y_pred):
   return sklearn.metrics.r2_score(y, y_pred)

In [10]:
def mean_squared_error(y, y_pred):
   return sklearn.metrics.mean_squared_error(y.to_numpy(), y_pred.to_numpy(), squared=False)

In [11]:
from sklearn.model_selection import KFold

columns = (
    [i + j for i in ["E", "STD", "R2", "RMSE"] for j in ["-train", "-test"]]
    + [f"w_{i}" for i in range(54)]
)

results = pd.DataFrame(columns=columns)

for train_index, test_index in KFold(shuffle=True).split(X_train):
    X_train_fold, y_train_fold = X_train.iloc[train_index], y_train.iloc[train_index]
    X_test_fold, y_test_fold = X_train.iloc[test_index], y_train.iloc[test_index]

    model = LinearRegression(learning_rate=0.05, epoches=100)
    model.fit(X_train_fold, y_train_fold)

    y_pred_train_fold = model.predict(X_train_fold)
    y_pred_test_fold = model.predict(X_test_fold)

    df_fold = pd.DataFrame(
        [
            [
                mean(y_train_fold, y_pred_train_fold),
                mean(y_test_fold, y_pred_test_fold),
                std(y_train_fold, y_pred_train_fold),
                std(y_test_fold, y_pred_test_fold),
                r2_score(y_train_fold, y_pred_train_fold),
                r2_score(y_test_fold, y_pred_test_fold),
                mean_squared_error(y_train_fold, y_pred_train_fold),
                mean_squared_error(y_test_fold, y_pred_test_fold),
            ]
            + [model.b]
            + list(model.W)
        ],
        columns=columns
    )

    results = pd.concat([results, df_fold], ignore_index=True)

results.apply(lambda s: s.apply('{0:.5f}'.format))

,E-train,E-test,STD-train,STD-test,R2-train,R2-test,RMSE-train,RMSE-test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17,w_18,w_19,w_20,w_21,w_22,w_23,w_24,w_25,w_26,w_27,w_28,w_29,w_30,w_31,w_32,w_33,w_34,w_35,w_36,w_37,w_38,w_39,w_40,w_41,w_42,w_43,w_44,w_45,w_46,w_47,w_48,w_49,w_50,w_51,w_52,w_53
0,0.00036,-0.45990,30.88217,21.77325,0.29973,0.44266,30.88217,21.77810,7.41459,0.25640,-0.65686,-1.48689,-0.15434,0.44615,-0.33656,0.62397,0.25574,1.95177,-0.22633,-0.80637,1.72551,1.47556,-0.21294,0.07610,-0.08884,0.80352,2.39044,1.29459,-0.77667,0.61794,-0.68055,-1.07203,-0.85510,0.35510,0.04818,0.15056,-0.23302,1.13501,0.04848,9.15733,2.99342,-0.87867,7.51003,-4.47828,0.06347,2.77407,0.40658,0.82160,-0.65612,-0.69044,-0.59439,-0.30286,-0.29534,-0.26857,-0.59869,-0.44051,0.06687,-0.21756,0.13057,-0.30409,-0.15322,-0.37699
1,-0.00000,-0.21614,29.42222,28.68304,0.31806,0.32643,29.42222,28.68386,7.36612,0.12848,-0.65001,-1.35262,-0.10029,1.85052,1.11223,1.65648,0.45614,1.33740,-1.03155,-0.18671,1.80441,2.66114,-0.56273,0.00181,-0.25914,0.97069,2.31359,0.34124,-2.33756,-1.02724,-0.25855,-1.39182,0.20276,-0.50007,-0.76174,0.73538,-0.21638,-0.42876,-0.52606,9.90487,2.18787,-0.04449,6.43890,-4.34978,0.07233,1.68847,-0.29093,0.75685,0.27935,0.26542,0.27766,0.72306,0.74667,0.83467,0.50892,-0.19123,0.23862,0.10661,0.49726,0.24685,0.17009,-0.07754
2,0.00062,0.74001,27.98698,34.70742,0.33375,0.24498,27.98698,34.71531,7.17439,-1.38806,-0.55787,-0.63755,-0.02566,-0.19269,-1.04179,0.10889,-1.31715,1.61051,-0.13231,-0.51618,2.04270,3.15233,0.25559,0.01403,0.85395,-0.39194,3.16147,-0.78326,0.04480,0.83026,0.07402,0.02596,1.23977,-0.46907,-1.12271,0.17028,-0.29088,-0.60210,-0.14146,9.75202,2.11612,-1.21772,6.22893,-4.11772,0.13681,5.62408,1.48686,0.92256,-0.18877,-0.28130,-0.42574,0.14334,0.01165,0.21076,-0.17994,-0.17808,0.10947,-0.11741,0.37876,0.10992,0.07922,-0.36269
3,0.00047,-0.40980,29.49900,28.34980,0.32568,0.29308,29.49900,28.35276,7.40448,0.23341,-0.45899,-1.65627,0.02999,-0.51763,0.61345,-0.18200,0.34415,0.33444,-1.82668,1.15859,1.75290,3.51539,0.11891,0.15622,0.40061,1.62918,2.09263,0.60044,0.38041,-0.19653,-0.03911,-1.01376,-0.02965,-0.16014,-1.41977,1.13912,-0.57676,-1.43832,-0.25082,10.44210,2.29567,-0.31532,6.76139,-4.23332,-0.01734,2.26848,0.54101,0.69534,-0.33076,-0.31471,-0.34770,-0.05883,-0.22276,-0.03776,-0.41948,0.33406,0.70409,0.32912,0.66720,0.26010,0.47760,0.29573
4,0.00022,0.22046,28.28943,32.89728,0.33400,0.27476,28.28943,32.89802,7.27863,0.01361,-0.61643,-1.42658,-0.08325,1.16635,1.65956,0.89175,0.10902,0.89547,-1.43842,0.19709,2.55808,2.46346,-2.05199,-0.30997,0.28029,0.72287,3.09727,-0.45336,-1.44910,-1.48091,-0.96668,-0.39204,1.67692,0.54654,0.24811,0.34475,0.06930,-0.65956,-0.10963,10.48440,2.13842,-0.25698,6.61497,-4.05523,-0.01450,2.17443,-0.65158,0.82766,-0.24786,-0.31329,-0.11730,0.28824,0.04799,-0.01497,-0.39495,-0.85232,-0.50337,-0.77771,-0.52136,-1.01296,-0.92135,-1.03841
